In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import cv2
import numpy as np
import tensorflow as tf
from tracker import Object_Tracker
from ultralytics import YOLO
import random
from deep_sort import generate_detections as gdet
import time

In [2]:
tracker= Object_Tracker()

In [3]:
model = YOLO('yolov8n.pt')

In [4]:
data_dir= 'C:\\dataset2014\\results'

In [5]:
all_items = os.listdir(data_dir)
categories = [item for item in all_items if os.path.isdir(os.path.join(data_dir, item))]
print(categories)

['badWeather', 'baseline', 'cameraJitter', 'dynamicBackground', 'intermittentObjectMotion', 'lowFramerate', 'nightVideos', 'PTZ', 'shadow', 'thermal', 'turbulence']


In [6]:
img=  'C:\\dataset2014\\results\\badWeather\\skating\\in000001.jpg'
img= cv2.imread(img)
results= model(img)

names=results[0].names


0: 640x640 1 car, 125.9ms
Speed: 5.0ms preprocess, 125.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


In [16]:
names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [7]:
img.shape

(320, 320, 3)

In [8]:
colors = [(random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for j in range(10)]

detection_threshold = 0.5

In [9]:
pwd

'C:\\Users\\Toqa Alaa\\deep_learning\\Real-time-object-tracking'

In [10]:
def non_max_suppression(detections,  threshold):

    if len(detections) == 0:
        return [], [], []

    boxes= [box[0:4] for box in detections]
    scores= [box[4].item() for box in detections]
    classes= [box[5] for box in detections]
    
    boxes= np.array(boxes)
    scores= np.array(scores)
    classes= np.array(classes)

    # Initialize an empty list to store the selected boxes.
    selected_boxes = []
    selected_scores= []
    selected_classes= []

    # Sort the boxes by their confidence scores in descending order.
    sorted_indices = np.argsort(scores)[::-1]
    boxes = boxes[sorted_indices]
    scores = scores[sorted_indices]
    classes= classes[sorted_indices]

    while len(boxes) > 0:
        # Select the box with the highest confidence score and add it to the selected list.
        selected_boxes.append(boxes[0])
        selected_scores.append(scores[0])
        selected_classes.append(classes[0])

        # Calculate the IoU (Intersection over Union) between the selected box and the remaining boxes.
        iou = calculate_iou(selected_boxes[-1], boxes[1:])

        # Filter out the boxes with IoU greater than or equal to the threshold.
        mask = iou < threshold
        boxes = boxes[1:][mask]
        scores = scores[1:][mask]

    return selected_boxes, selected_scores, selected_classes

In [11]:
def calculate_iou(box1, boxes2):
    x1, y1, x2, y2 = box1
    x1 = np.maximum(x1, boxes2[:, 0])
    y1 = np.maximum(y1, boxes2[:, 1])
    x2 = np.minimum(x2, boxes2[:, 2])
    y2 = np.minimum(y2, boxes2[:, 3])

    intersection_area = np.maximum(0, x2 - x1 + 1) * np.maximum(0, y2 - y1 + 1)
    box1_area = (x2 - x1 + 1) * (y2 - y1 + 1)
    boxes2_area = (boxes2[:, 2] - boxes2[:, 0] + 1) * (boxes2[:, 3] - boxes2[:, 1] + 1)

    iou = intersection_area / (box1_area + boxes2_area - intersection_area)
    return iou

In [12]:
data_path='C:\\dataset2014\\results\\baseline\\highway'
video_path= 'traffic_video.mp4'

In [13]:
vid = cv2.VideoCapture(video_path) 

In [14]:
frames=[]

In [15]:
while(True):
# start_time = time.time()

# for img in os.listdir(data_path):
#     frame = cv2.imread(os.path.join(data_path, img))
    _, frame = vid.read()
    # Perform object detection using YOLO on the current frame
    results = model(frame)  

    # Update the tracks using the DeepSort tracker
    for result in results:
        detections = []
        

        # Creating the detections array
        for r in result.boxes.data:
            x1, y1, x2, y2, score, class_id = r
            x1 = int(x1)
            x2 = int(x2)
            y1 = int(y1)
            y2 = int(y2)

            class_id = int(class_id)
            class_name= names[class_id]
            if score>=0.3:
                detections.append([x1, y1, x2, y2, score, class_name])
            
        
       
        boxes, score, classes= non_max_suppression(detections, 0.5)
        detections=[]
        for i in range(len(boxes)):
            b= []
            b.append(boxes[i][0])
            b.append(boxes[i][1])
            b.append(boxes[i][2])
            b.append(boxes[i][3])
            b.append(score[i])
            b.append(classes[i])
            
            
            detections.append(b)
        
        print(detections)
    
        tracker.update(frame, detections)
        # Updating the tracks with the new frame
        for track in tracker.tracks:
            bbox = track.bbox
            x1, y1, x2, y2 = bbox
            track_id = track.track_id
            
            

            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (colors[track_id % len(colors)]), 3)

            # Define the text you want to add
            text = track.class_name

            # Define the position where you want to place the text (adjust coordinates as needed)
            text_x = int(x1)  # You can adjust the x-coordinate as needed
            text_y = int(y1) - 10  # You can adjust the y-coordinate as needed

            # Get the color of the rectangle
            rectangle_color = colors[track_id % len(colors)]

            # Get the size of the text to calculate the size of the background rectangle
            (text_width, text_height), baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            background_width = text_width + 10
            background_height = text_height + 10

            # Define the position of the background rectangle
            background_x = text_x
            background_y = text_y - text_height

            # Draw the background rectangle
            cv2.rectangle(frame, (background_x, background_y), (background_x + background_width, background_y + background_height), rectangle_color, -1)

            # Define the font and font scale
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.5

            # Define the color of the text (you can change this if needed)
            text_color = (255, 255, 255)  # White color in BGR

            # Add the text to the frame
            cv2.putText(frame, text, (text_x, text_y), font, font_scale, text_color, 1, cv2.LINE_AA)
        # Save the frame with tracking results
        cv2.imshow('img', frame)
        
        frames.append(frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()
vid.release()


0: 384x640 1 car, 77.8ms
Speed: 11.0ms preprocess, 77.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


[[607, 307, 633, 336, 0.6374568939208984, 'car']]



0: 384x640 1 car, 82.8ms
Speed: 3.0ms preprocess, 82.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 76.8ms
Speed: 2.0ms preprocess, 76.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 70.8ms


[[607, 307, 633, 336, 0.6399165391921997, 'car']]
[[606, 307, 633, 336, 0.6196421980857849, 'car']]


Speed: 4.0ms preprocess, 70.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 56.8ms
Speed: 3.0ms preprocess, 56.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 cars, 60.8ms
Speed: 2.0ms preprocess, 60.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



...

0: 384x640 5 cars, 66.8ms
Speed: 3.0ms preprocess, 66.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 61.8ms
Speed: 3.0ms preprocess, 61.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



[[527, 355, 570, 400, 0.6638322472572327, 'car'], [822, 388, 881, 422, 0.5744150280952454, 'car'], [561, 328, 606, 358, 0.49475204944610596, 'car'], [607, 306, 634, 326, 0.4870505928993225, 'car']]
[[520, 358, 578, 407, 0.6349712014198303, 'car'], [818, 385, 876, 419, 0.5620149970054626, 'car'], [559, 332, 592, 358, 0.49176356196403503, 'car'], [606, 306, 634, 326, 0.471804141998291, 'car']]


0: 384x640 5 cars, 68.8ms
Speed: 4.0ms preprocess, 68.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 65.8ms
Speed: 2.0ms preprocess, 65.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)



[[517, 361, 590, 409, 0.8075671792030334, 'car'], [557, 333, 608, 361, 0.40263187885284424, 'car'], [815, 382, 868, 416, 0.3960164487361908, 'car'], [606, 307, 633, 326, 0.37846502661705017, 'car']]
[[513, 363, 591, 411, 0.6226015090942383, 'car'], [814, 379, 869, 414, 0.41244497895240784, 'car'], [554, 338, 590, 360, 0.32361090183258057, 'car'], [605, 306, 631, 327, 0.3160513937473297, 'car']]



KeyboardInterrupt



In [ ]:
output_file = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
frame_rate = 30  # Adjust this to your desired frame rate
width, height = frames[0].shape[1], frames[0].shape[0]  # Use dimensions from the first frame

# Create VideoWriter object
out = cv2.VideoWriter(output_file, fourcc, frame_rate, (width, height))

# Loop through the array of frames and write them to the video
for frame in frames:
    out.write(frame)

# Release the VideoWriter
out.release()

# Optionally, you can also destroy any OpenCV windows that might be open
cv2.destroyAllWindows()